# Contrastive Anamoly Explanations generation via Python SDK

Likelihood compensation (LC) is a framework for explaining observed deviations of a black-box regression model $y=f(\mathbf{x})$, where $y$ is a real-valued target variable and $\mathbf{x}$ is a multivariate numerical vector as the input.

### Problem Statement

In the field of XAI (explainable AI), LC falls into the category of local explanation. However, it addresses a unique problem setting that differs from most of XAI approaches. In words, the task is as follows:

With a $\mathbf{x}=\mathbf{x}^t$ observed, a black-box model predicted the target to be $f(\mathbf{x}^t)$. However, the actual observation $y = y^t$ significantly deviated from the prediction. Which input variables are most responsible for the deviation and how?

### Difference from existing XAI methods

We need to keep in mind that LC needs both $\mathbf{x}=\mathbf{x}^t$ and $y=y^t$. This setting differs from most of the XAI approaches for regression, such as LIME and Shapley values, which typically require only $\mathbf{x}=\mathbf{x}^t$ to explain local properties of $f(\mathbf{x})$. In contrast, LC is to explain the deviation:

   


Most of the existing XAI approaches to regression:
    
Given $\mathbf{x}=\mathbf{x}^t$, explain the black-box function $f(\mathbf{x})$ in the vicinity of $\mathbf{x}=\mathbf{x}^t$.

LC:
    
Given $\mathbf{x}=\mathbf{x}^t$ AND $y=y^t$, locally explain where a large deviation $y^t - f(\mathbf{x}^t)$ came from.


**Note** : **This notebook works only with Default python3.9 and Default Python 3.8 environments in case of WatsonStudio and CPD**

## Setup

## Package installation

In [ ]:
!pip install --upgrade pyspark==3.0.2 --no-cache | tail -n 1
!pip install matplotlib
!pip install seaborn
!pip install --upgrade ibm-metrics-plugin --no-cache | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1

## Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale). 

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam/users) section of the Cloud console. From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

**NOTE**: You can also get OpenScale `API_KEY` using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html)

How to get api key using console: \
\
&emsp; bx login --sso  
&emsp; bx iam api-key-create 'my_key' 

In [ ]:
CLOUD_API_KEY = ""

#datamart_id is same as the Watson Openscale service instance id
datamart_id = ""

### Initialize spark session

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pandas as pd

sparkconf = SparkConf().setMaster("local[*]")
spark = SparkSession.builder.appName("TestMetricFramework").config(conf=sparkconf).getOrCreate()
spark.sparkContext._conf.getAll()

### Import dataset

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=123)

### Building a black-box prediction function

For end-to-end demonstration purposes, we train a Linear Regression model and think of it as a black-box prediction function once trained. We will not use any information on the internal parameters of the model and the training data later on.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_train, y_train)

## Configure Openscale

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [ ]:
from ibm_watson_openscale import APIClient as OpenScaleAPIClient
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator, BearerTokenAuthenticator

wos_authenticator = IAMAuthenticator(
    apikey=CLOUD_API_KEY
)

client = OpenScaleAPIClient(
    authenticator=wos_authenticator,
    service_instance_id=datamart_id
)
client.version

### Detecting anomalies/outliers

lc comes with anomaly_score(), a function for computing anomaly score for observed y values. The anomaly detection model requires a model parameter named sigma_yf, which is the standard deviation of the deviation $y - f(\mathbf{x})$

In [ ]:
sigma_yf = (y_test - model.predict(X_test)).std()
print('predictive std dev. sigma_yf={}'.format(sigma_yf))

We use anomaly_score() to compute the anomaly score. Let us focus on the sample with the highest anomaly score.

In [ ]:
%%time
import numpy as np
from ibm_metrics_plugin.metrics.explainability.explainers.contrastive_anamoly import lc_util as util
a = util.anomaly_score(X_test,y_test,model.predict,sigma_yf=sigma_yf)

import matplotlib.pyplot as plt
import seaborn as sb; sb.set()
fig,ax=plt.subplots(figsize=(6,3))
x = np.arange(0,len(a))
ax.scatter(x,a,color='black',alpha=0.5)
ax.set_xlabel('sample index',fontsize=18)
ax.set_ylabel('anomaly score',fontsize=18)
fig.tight_layout()

In [ ]:
idx_argmax = np.argmax(a)
print('max value a[{}]={}'.format(idx_argmax,a[idx_argmax]))

## Computing LC score for detected outliers

We pick the sample with the highest anomaly score ($n=21$) and compute the LC score.

In [ ]:
idx_argmax = np.argmax(a)
print('maximum anomaly score: a[{}]={}'.format(idx_argmax,a[idx_argmax]))

# Pick the sample with the highest anomaly score
x_outlier = X_test[idx_argmax,:]
y_outlier = y_test[idx_argmax]

### Define the input data row for which contrastive anamoly explanation needs to be generated

As identified above, the test sample at index 21 is an anamoly. 

In [ ]:
test_data = np.append(x_outlier, y_outlier)
cols = list(load_boston().feature_names)
cols.append("PRICE")
test_df_in = pd.DataFrame([test_data], columns=cols)
testDF_spark = spark.createDataFrame(test_df_in)
testDF_spark.show()

### Set configuration

It is important to generate lc_stats when the input data contains NaN's so that the accuracy of the explanation is not impacted.

The below method uses some learnings made on the training data to impute NaNs in the test data. Hence, it is mandatory to provide training data
to compute lc_stats

Uncomment the below cell only if the test data contains NaN.

In [ ]:
lc_stats = None
'''from ibm_metrics_plugin.metrics.explainability.entity.training_stats import TrainingStats
feature_cols = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"]
training_data_info = {
            "label_column": "PRICE",
            "feature_columns":  feature_cols,
            "problem_type": "regression"
        }
df_boston = pd.DataFrame(X,columns=feature_cols)
df_boston['PRICE'] = pd.Series(y)
training_stats = TrainingStats(df_boston, training_data_info)
lc_stats = training_stats.compute_lc_stats()'''

In [ ]:
#Set lc_inputs
X_scales=X_test.std(axis=0)
lc_inputs = {
                    "sigma_yf": sigma_yf,
                    "x_scales": X_scales,
                    "lc_stats": lc_stats
    
                }

In [ ]:
#Set metrics plugin level configuration
configuration = {}
configuration['configuration'] = {
            "problem_type": "regression",
            "label_column": "PRICE",
            "prediction": "prediction",
            "input_data_type": "structured",
            "feature_columns": ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT"],
            "explainability": {
                "metrics_configuration":{
                "contrastive_anamoly": lc_inputs
                }
            }
}

In [ ]:
results = client.ai_metrics.compute_metrics(spark=spark, configuration=configuration, data_frame=testDF_spark, scoring_fn=model.predict)

### Generate Contrastive Anamoly Explanations

In [ ]:
#Read output and print
import json
metrics = results.get("metrics_result")
lc_metrics = {}

if(metrics.get("explainability")):
        explain_metrics = metrics.get("explainability")
        lc_metrics = explain_metrics.get("contrastive_anamoly")
        
if not lc_metrics:
    print("unable to compute lc metrics")
else:
    print(lc_metrics)

### Plotting LC scores

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))  
delta = list(lc_metrics[0].values())
cols = lc_metrics[0].keys()
ax.bar(cols,delta)
ax.tick_params(axis='x', rotation=90) 
ax.set_title('LC score $\delta$')
fig.tight_layout()

### Understanding computed LC score

The computed LC score suggests:

1) If RM had been larger by 0.5, the fit would have been better (and the anomaly score would have been lower). That is, RM is too small.

2) If NOX had been smaller by 1.4, the fit would have been better (and the anomaly score would have been lower). That is, NOX is a bit too large.

3) If DIS had been smaller by 0.19, the fit would have been better (and the anomaly score would have been lower). That is, DIS is large.


### Shutdown spark

In [ ]:
spark.stop()